In [139]:
from gensim.models import Word2Vec
import pandas as pd
import nltk
from tqdm import tqdm
import numpy as np

In [2]:
Cell_Phones_and_Accessories = pd.read_json('./data/reviews_Cell_Phones_and_Accessories_5.json', lines=True)

In [95]:
stopwords=[]
with open('stopwords.txt') as file:
    for line in file.readlines():
        stopwords.append(line.strip())

In [109]:
sentences = []
with tqdm(total = len(Cell_Phones_and_Accessories)) as pbar:
    for review in Cell_Phones_and_Accessories.iloc[:,3]:
        review = nltk.sent_tokenize(review)
        for i in range(len(review)):
            review[i] = review[i].lower()
            for punc in string.punctuation:
                review[i] = review[i].replace(punc, ' ')
        sentences.append(review)
        pbar.update(1)

100%|██████████| 194439/194439 [01:36<00:00, 2008.11it/s]


In [110]:
s = []
for sentence in sentences:
    for ss in sentence:
        s.append(ss)
print len(s)

1001122


In [111]:
for i in range(len(s)):
    s[i] = nltk.word_tokenize(s[i])

In [115]:
for i in range(len(s)):
    new_sent = []
    for word in s[i]:
        if word not in stopwords and len(word)>2:
            new_sent.append(word)
    s[i] = new_sent
print s[0:2]

[[u'look', u'good', u'stick', u'good'], [u'just', u'don', u'like', u'rounded', u'shape', u'always', u'bumping', u'siri', u'kept', u'popping', u'irritating']]


In [116]:
model = Word2Vec(s, size=100, window=5, min_count=5, workers=4)

In [117]:
model.save('word2vec-model')

In [119]:
keys = model.wv.vocab.keys()

In [120]:
key_dict = {}
for key in keys:
    key_dict[key] = 0
for sentence in s:
    for word in sentence:
        if key_dict.has_key(word):
            key_dict[word] += 1
key_list = sorted(key_dict.items(), key=lambda x:x[1], reverse=True)

In [174]:
features = np.array(key_list[:1000])[:,0]
feature_matrix = []
with tqdm(total = len(Cell_Phones_and_Accessories)) as pbar:
    for review in Cell_Phones_and_Accessories.iloc[:,3]:
        review = review.lower()
        for punc in string.punctuation:
            review = review.replace(punc, ' ')
        review = nltk.word_tokenize(review)
        feature_vector = np.zeros(1000)
        feature_dict = {}
        for feature in features:
            feature_dict[feature] = 0
        for word in review:
            similar_words = []
            if word not in stopwords and len(word)>2:
                if model.wv.vocab.has_key(word):
                    similar_words = model.most_similar(word)
            for similar_word_pair in similar_words:
                similar_word = u''+similar_word_pair[0]
                if feature_dict.has_key(similar_word):
                    feature_dict[similar_word] += 1
        for i in range(len(features)):
            feature_vector[i] = feature_dict[features[i]]
        feature_matrix.append(feature_vector)
        pbar.update(1)

100%|██████████| 194439/194439 [2:47:30<00:00, 35.41it/s]


In [175]:
np.sum(feature_matrix)

29341446.0

In [176]:
np.save('word2vec_feature_matrix',feature_matrix)

In [178]:
np.savetxt('word2vec_feature_matrix.csv', feature_matrix, delimiter = ',')  